## 학습정리

### 13장. 나이브 베이즈

#### 13.1 바보 스팸 필터
* ex. 바보 스팸 필터 (스팸 - 비트코인이라는 단어를 포함)
    * 스팸 메시지 중 50%가 비트코인이라는 단어를 포함, 스팸이 아닌 메시지중 1%가 비트코인이라는 단어를 포함 
    * $P(S|B) = \frac{P(B|S)}{P(B|S)+P(B|S*)}$ : 0.5/0.51 = 98%

#### 13.2 조금 더 똑똑한 스팸 필터
* 더 많은 단어($w_1,...w_n$)가 주어졌을 때
    * 단어 $w_i$가 메시지에 포함되는 경우를 $X_i$로 나타냄
    * $P(X_i|S)$ : 스팸 메시지에 i번째 단어가 포함되어 있는 확률
    * $P(X_i|S*)$ : 스팸이 아닌 메시지에 i번째 단어가 포함되어 있는 확률

* 나이브 베이즈(Naive)의 핵심
    * 극단적인 가정 : 메시지의 스팸여부가 주어졌을 때 각 단어의 존재(또는 부재)가 서로 조건부 독립적이다
        * $P(X_1=x_1,...,X_n=x_n|S) = P(X_1=x_1|S)\times...\times P(X_n=x_n|S)$
            * $P(X|S) = \frac{P(S|X)}{P(S|X)+P(S*|X)}$ 
        * 두 단어가 동시에 등장하지 않는경우의 확률을 무시, 하지만 성능이 뛰어남
    * 주의
        * 실제 구현 시 끊임없이 확률 값을 곱하는 것을 피해야함
            * 언더플로(underflow) : 컴퓨터가 0에 가까운 부동소수점(floating point number)를 제대로 처리 못함
            * 문제를 해결하기 위해 log를 사용
        * $P(X_i|S/S*)$ : 스팸안/스팸이 아닌 메시지에 단어 $w_i$가 포함될 확률
            * 주어진 학습 데이터에서 '데이터'라는 단어가 스팸이 아닌 메시지에만 포함되어 있다면 분류기는 '데이터'가 들어간 메시지를 항상 스팸이 아니라고 예측한다.
                * 문제를 해결하기 위해서 스무딩(smoothing, 평활화)활용 : 가짜 빈도수(pseudocount)k를 결정
                    * 0이 아닌 아주 작은 확률값으로 설정
            
#### 13.3 구현하기
1. 메시지를 단어 단위로 잘라주는 함수 생성
    * 메시지를 모두 소문자로 통일
    * re.findall을 사용해서 문자,숫자,혹은 아포스트로피(apostrophe)가 포함된 모든 '단어'를 추출
    * 집합을 사용해서 중복되는 단어를 제거
    
2. 학습 데이터를 위한 타입 정의
    * 학습 데이터의 단어, 빈도, 레이블을 관리하기 위해 클래스로 생성
        * 관례 : 스팸이 아닌 메일을 햄(ham)으로 표기 
        * 생성자 : 가짜 빈도수를 유일한 인자로 받아 확률 계산에 활용
            * 각 단어가 얼마나 자주 등장했는지 기록하기 위한 빈 집합 생성

3. 메시지를 사용해서 학습할 수 있도록 메서드 생성
    * 두 메시지의 빈도를 기록
    * 각 메시지를 단어 기준으로 나눔
    * 각 단어별로 메시지의 종류에 따라 token count

4. 프라이빗 도우미 함수 생성
    * 메시지 안의 모든 단어마다 P(해당 단어|스팸),P(해당 단어|햄) 값이 필요 

5. predict 메서드 함수 생성
    * log확률값의 덧셈으로 구성
    
#### 13.4 모델 검증하기
* 생성된 모델이 정상 동작 하는지 유닛 테스트를 작성하여 확인
    * 개수를 제대로 맞추는지
    * 예측이 맞는지

#### 13.5 모델 사용하기
* SpamAssasin public corpus 데이터

* 스팸 메시지를 대표하는 단어 : sale, mortgage, money, rate ,,
* 햄 메시지를 대표하는 단어 : spambayes, users, apt, perl ..

* 더 좋은 성능을 얻는 방법  
    * 더 많은 학습 데이터를 구하는 것
    * 메시지의 제목 뿐 아니라 내용을 활용(헤더)
    * 단어의 최소 빈도수를 설정해서 기준보다 적게 나온 단어를 무시
    * 동의어를 고려 : 간단한 어간 추출기(stemmer)를 분류기에 추가, 동의어를 같은 그룹으로 묶어줌
        * 이미 구현되어 있는 Porter stemmer를 사용
    * 모델의 변수로 '메시지가 단어w를 포함한다'의 꼴만 사용했지만 다른 꼴도 사용가능
        * ex. 숫자가 포함되어 있는 경우 문자열을 반환하도록 함
        
#### 13.6 더 공부해 보고 싶다면
* 폴 그레이엄(Paul Graham) - A Plan for Spam, Better Bayesian Filtering
* scikit-learn : BernoulliNB(베르누이 나이브 베이즈)모델 등 다양한 모델 제공



## code

In [1]:
from typing import Set
import re

# 단어 단위로 잘라주는 함수
def tokenize(text: str) -> Set[str] :
    text = text.lower()   # 소문자
    all_words = re.findall("[a-z0-9]+", text) # 단어 추출
    return set(all_words) # 중복 제거 


In [2]:
from typing import NamedTuple

# 데이터 타입 정의 
class Message(NamedTuple) :
    text : str
    is_spam : bool


In [11]:
from typing import List,Tuple,Dict,Iterable
import math
from collections import defaultdict
# 학습 데이터의 단어, 빈도, 레이블을 관리하기 위해 클래스로 생성
class NaiveBayesClassifier :
    def __init__(self, k:float = 0.5) -> None :
        
        self.k = k # 스무딩 인수
        self.tokens : Set[str] = set()
        self.token_spam_counts : Dict[str, int] = defaultdict(int) 
        self.token_ham_counts : Dict[str, int] = defaultdict(int)
        self.spam_messages = self.ham_messages = 0
    # 메시지를 사용해서 학습할 수 있도록 메서드 생성
    # 두 메시지의 빈도를 기록
    # 각 메시지를 단어 기준으로 나눔
    # 각 단어별로 메시지의 종류에 따라 token count

    def train(self, messages : Iterable[Message]) -> None :
        for message in messages :
            # 메시지의 수를 증가시킴
            if message.is_spam :
                self.spam_messages += 1
            else :
                self.ham_messages += 1
    
            # 단어의 빈도를 증가시킨다.
            for token in tokenize(message.text) :
                self.tokens.add(token)
                if message.is_spam :
                    self.token_spam_counts[token] += 1
                else :
                    self.token_ham_counts[token] += 1
    
    
    # 프라이빗 도우미 함수
    def _probabilities(self, token : str) -> Tuple[float, float] :
        """P(단어 | 스팸)과 P(단어|햄)을 반환"""
        spam = self.token_spam_counts[token]
        ham = self.token_ham_counts[token]
    
        p_token_spam = (spam + self.k) / (self.spam_messages + 2* self.k)
        p_token_ham = (ham + self.k) / (self.ham_messages + 2 * self.k)
    
        return p_token_spam, p_token_ham 
    
    
    # 예측 수행 메서드
    # 분류기 : 스팸일 확률을 확인 
    def predict(self, text : str) -> float :
        text_tokens = tokenize(text)
        log_prob_if_spam = log_prob_if_ham = 0.0
    
        # 모든 메시지 안의 각 단어를 순회
        for token in self.tokens :
            prob_if_spam, prob_if_ham = self._probabilities(token)
        
            # 만약 *token*이 메시지에 나온다면
            # 단어가 등장할 로그 확률값을 더함
            if token in text_tokens :
                log_prob_if_spam += math.log(prob_if_spam)
                log_prob_if_ham += math.log(prob_if_ham)
        
            # 그게 아니라면 단어가 등장하지 않을 로그 확률을 더함
            # 등장하지 않을 확률 : log(1-등장할 확률)
            else :
                log_prob_if_spam += math.log(1.0 - prob_if_spam)
                log_prob_if_ham += math.log(1.0 - prob_if_ham)
    
        prob_if_spam = math.exp(log_prob_if_spam)
        prob_if_ham = math.exp(log_prob_if_ham)
        return prob_if_spam / (prob_if_spam + prob_if_ham)                
    

In [13]:
# 확인해보기
messages = [Message("spam rules", is_spam = True),
           Message("ham rules", is_spam = False),
           Message("hello ham", is_spam = False)]

model = NaiveBayesClassifier(k = 0.5)
model.train(messages)

In [15]:
print(model.tokens)
print(model.spam_messages)
print(model.ham_messages)
print(model.token_spam_counts)
print(model.token_ham_counts)

{'spam', 'hello', 'ham', 'rules'}
1
2
defaultdict(<class 'int'>, {'spam': 1, 'rules': 1})
defaultdict(<class 'int'>, {'ham': 2, 'rules': 1, 'hello': 1})


In [19]:
text = ("Hello spam")
model.predict(text)

0.8350515463917525